# A tour of PyCIEMSS interfaces and functionality

### Load dependencies and interfaces

In [1]:
import os
import pyciemss
import torch

### Select models and data

In [2]:
MODEL_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/models/"
DATA_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration/main/data/datasets/"

model1 = os.path.join(MODEL_PATH, "SEIRHD_NPI_Type1_petrinet.json")
model2 = os.path.join(MODEL_PATH, "SEIRHD_NPI_Type2_petrinet.json")
model3 = os.path.join(MODEL_PATH, "SIR_stockflow.json")

dataset1 = os.path.join(DATA_PATH, "traditional.csv")

### Set parameters for sampling

In [3]:
start_time = 0.0
end_time = 100.
logging_step_size = 10.0
num_samples = 3

## Sample interface
Take `num_samples` number of samples from the (prior) distribution invoked by the chosen model.

### Sample from model 1

In [4]:
result1 = pyciemss.sample(model1, end_time, logging_step_size, num_samples, start_time=start_time)
result1["unprocessed_result"]

{'persistent_beta_c': tensor([0.3809, 0.5298, 0.1499]),
 'persistent_kappa': tensor([0.6597, 0.6434, 0.2495]),
 'persistent_gamma': tensor([0.1145, 0.4444, 0.2719]),
 'persistent_hosp': tensor([0.0702, 0.0105, 0.0507]),
 'persistent_death_hosp': tensor([0.0433, 0.0331, 0.0370]),
 'persistent_I0': tensor([4.0432, 9.0951, 4.8514]),
 'D_state': tensor([[7.6402e-02, 9.8125e-01, 9.9774e+00, 9.8816e+01, 9.4240e+02, 6.9622e+03,
          2.4417e+04, 4.2327e+04, 5.2224e+04],
         [1.6649e-02, 6.4542e-02, 1.5898e-01, 3.4102e-01, 6.9136e-01, 1.3654e+00,
          2.6618e+00, 5.1539e+00, 9.9383e+00],
         [4.8411e-02, 1.4036e-01, 2.3060e-01, 3.1252e-01, 3.8606e-01, 4.5197e-01,
          5.1103e-01, 5.6393e-01, 6.1133e-01]]),
 'E_state': tensor([[1.9795e+02, 1.9627e+03, 1.9429e+04, 1.8869e+05, 1.5353e+06, 4.1934e+06,
          1.5152e+06, 2.4173e+05, 3.6683e+04],
         [6.1207e+01, 1.1779e+02, 2.2667e+02, 4.3617e+02, 8.3918e+02, 1.6141e+03,
          3.1029e+03, 5.9584e+03, 9.9665e+03],

In [5]:
result1['data'].head()

,timepoint_id,sample_id,persistent_beta_c_param,persistent_kappa_param,persistent_gamma_param,persistent_hosp_param,persistent_death_hosp_param,persistent_I0_param,D_state_state,E_state_state,H_state_state,I_state_state,R_state_state,S_state_state,infected_observable_state,exposed_observable_state,hospitalized_observable_state,dead_observable_state
0,0,0,0.38094,0.659716,0.114534,0.070212,0.043332,4.043168,0.076402,1.979472e+02,2.628090,1.437959e+02,59.838490,19339634.0,1.437959e+02,1.979472e+02,2.628090,0.076402
1,1,0,0.38094,0.659716,0.114534,0.070212,0.043332,4.043168,0.981251,1.962729e+03,26.705475,1.426573e+03,675.190979,19335944.0,1.426573e+03,1.962729e+03,26.705475,0.981251
2,2,0,0.38094,0.659716,0.114534,0.070212,0.043332,4.043168,9.977351,1.942884e+04,264.788025,1.413366e+04,6775.909180,19299422.0,1.413366e+04,1.942884e+04,264.788025,9.977351
3,3,0,0.38094,0.659716,0.114534,0.070212,0.043332,4.043168,98.816376,1.886931e+05,2605.041260,1.384353e+05,66878.148438,18943322.0,1.384353e+05,1.886931e+05,2605.041260,98.816376
4,4,0,0.38094,0.659716,0.114534,0.070212,0.043332,4.043168,942.400452,1.535287e+06,23889.230469,1.218634e+06,625165.500000,15936108.0,1.218634e+06,1.535287e+06,23889.230469,942.400452


### Sample from model 2

In [6]:
result2 = pyciemss.sample(model2, end_time, logging_step_size, num_samples, start_time=start_time)
result2['data'].head()

,timepoint_id,sample_id,persistent_beta_c_param,persistent_beta_nc_param,persistent_kappa_param,persistent_gamma_param,persistent_hosp_param,persistent_death_hosp_param,persistent_I0_param,D_state_state,E_state_state,H_state_state,I_state_state,R_state_state,S_state_state,infected_observable_state,exposed_observable_state,hospitalized_observable_state,dead_observable_state
0,0,0,0.639174,0.100016,0.778697,0.385849,0.027484,0.057203,10.355591,0.088870,115.933159,1.737752,56.346195,117.926811,19339750.0,56.346195,115.933159,1.737752,0.088870
1,1,0,0.639174,0.100016,0.778697,0.385849,0.027484,0.057203,10.355591,0.508287,418.981750,6.563768,203.653931,555.046448,19338856.0,203.653931,418.981750,6.563768,0.508287
2,2,0,0.639174,0.100016,0.778697,0.385849,0.027484,0.057203,10.355591,2.038073,1513.907959,23.758307,735.909851,2134.967773,19335660.0,735.909851,1513.907959,23.758307,2.038073
3,3,0,0.639174,0.100016,0.778697,0.385849,0.027484,0.057203,10.355591,7.566987,5465.788086,85.822601,2657.507568,7842.259277,19323980.0,2657.507568,5465.788086,85.822601,7.566987
4,4,0,0.639174,0.100016,0.778697,0.385849,0.027484,0.057203,10.355591,27.521471,19676.136719,309.480621,9574.394531,28428.525391,19282024.0,9574.394531,19676.136719,309.480621,27.521471


## Ensemble Sample Interface
Sample from an ensemble of model 1 and model 2 

In [7]:
model_paths = [model1, model2]
solution_mappings = [lambda x : x, lambda x : x] # Conveniently, these two models operate on exactly the same state space, with the same names.

ensemble_result = pyciemss.ensemble_sample(model_paths, solution_mappings, end_time, logging_step_size, num_samples, start_time=start_time)
ensemble_result['data'].head()

,timepoint_id,sample_id,model_0/persistent_beta_c_param,model_0/persistent_kappa_param,model_0/persistent_gamma_param,model_0/persistent_hosp_param,model_0/persistent_death_hosp_param,model_0/persistent_I0_param,model_1/persistent_beta_c_param,model_1/persistent_beta_nc_param,...,model_0/H_state_state,model_0/I_state_state,model_0/R_state_state,model_0/S_state_state,model_1/D_state_state,model_1/E_state_state,model_1/H_state_state,model_1/I_state_state,model_1/R_state_state,model_1/S_state_state
0,0,0,0.429885,0.287685,0.411619,0.16625,0.070769,5.903922,0.204439,0.602292,...,3.958404,11.983437,50.645584,19339972.0,0.066212,7.336195,1.076627,14.994347,30.334200,19339996.0
1,1,0,0.429885,0.287685,0.411619,0.16625,0.070769,5.903922,0.204439,0.602292,...,3.106267,7.235049,89.750000,19339938.0,0.160400,2.686795,0.717982,6.409312,50.554142,19339996.0
2,2,0,0.429885,0.287685,0.411619,0.16625,0.070769,5.903922,0.204439,0.602292,...,1.968762,4.356900,113.891106,19339920.0,0.211181,1.069459,0.330637,2.580462,59.102703,19339996.0
3,3,0,0.429885,0.287685,0.411619,0.16625,0.070769,5.903922,0.204439,0.602292,...,1.198856,2.623672,128.507568,19339908.0,0.233411,0.428417,0.138439,1.034571,62.570225,19339996.0
4,4,0,0.429885,0.287685,0.411619,0.16625,0.070769,5.903922,0.204439,0.602292,...,0.723735,1.579944,137.320084,19339908.0,0.242578,0.171700,0.056290,0.414659,63.965023,19339996.0


## Calibrate interface
Calibrate a model to a dataset by mapping model state varibale or observables to columns in the dataset

In [8]:
data_mapping = {"Infected": "I"} # data_mapping = "column_name": "observable/state_variable"
num_iterations = 10
calibrated_results = pyciemss.calibrate(model1, dataset1, data_mapping=data_mapping, num_iterations=num_iterations)
parameter_estimates = calibrated_results["inferred_parameters"]
calibrated_results

{'inferred_parameters': AutoGuideList(
   (0): AutoDelta()
   (1): AutoLowRankMultivariateNormal()
 ),
 'loss': 245.3486876487732}

In [9]:
parameter_estimates()

{'persistent_beta_c': tensor(0.4892, grad_fn=<ExpandBackward0>),
 'persistent_kappa': tensor(0.3413, grad_fn=<ExpandBackward0>),
 'persistent_gamma': tensor(0.2778, grad_fn=<ExpandBackward0>),
 'persistent_hosp': tensor(0.1001, grad_fn=<ExpandBackward0>),
 'persistent_death_hosp': tensor(0.0555, grad_fn=<ExpandBackward0>),
 'persistent_I0': tensor(3.9059, grad_fn=<ExpandBackward0>)}

## Pass the parameter estimates to `sample` to sample from the calibrated model

In [10]:
calibrated_sample_results = pyciemss.sample(model1, end_time, logging_step_size, num_samples, 
                start_time=start_time, inferred_parameters=parameter_estimates)
calibrated_sample_results

{'data':     timepoint_id  sample_id  persistent_beta_c_param  persistent_kappa_param  \
 0              0          0                 0.435312                0.376931   
 1              1          0                 0.435312                0.376931   
 2              2          0                 0.435312                0.376931   
 3              3          0                 0.435312                0.376931   
 4              4          0                 0.435312                0.376931   
 5              5          0                 0.435312                0.376931   
 6              6          0                 0.435312                0.376931   
 7              7          0                 0.435312                0.376931   
 8              8          0                 0.435312                0.376931   
 9              0          1                 0.445362                0.398027   
 10             1          1                 0.445362                0.398027   
 11             2   

In [11]:
# TODO:
# - Add intervention example
# - Add examples for calibrate_ensemble and optimize interfaces as they become available
# - Plot results

## Sample interface with intervention

In [12]:
start_time = 0.0
end_time = 40.
logging_step_size = 1.0
num_samples = 5
result = pyciemss.sample(model3, end_time, logging_step_size, num_samples, start_time=start_time, 
                         static_parameter_interventions={torch.tensor(1.): {"p_cbeta": torch.tensor(0.35)}}, solver_method="euler")
result["data"]

,timepoint_id,sample_id,persistent_p_cbeta_param,persistent_p_tr_param,I_state_state,R_state_state,S_state_state
0,0,0,0.392797,9.763371,1.289981,0.102424,999.607605
1,1,0,0.392797,9.763371,1.608722,0.234548,999.156738
2,2,0,0.392797,9.763371,2.005967,0.399319,998.594727
3,3,0,0.392797,9.763371,2.500910,0.604778,997.894348
4,4,0,0.392797,9.763371,3.117360,0.860930,997.021729
...,...,...,...,...,...,...,...
190,34,4,0.270017,15.308768,507.518890,383.854675,109.626579
191,35,4,0.270017,15.308768,493.820404,417.006836,90.172890
192,36,4,0.270017,15.308768,477.132690,449.264191,74.603233
193,37,4,0.270017,15.308768,458.411438,480.431458,62.157207


### Optimize interface
Get infections below 300 individuals at 100 days for SIR model with minimum change to current value for intervention parameter

In [13]:
from pyciemss.ouu.qoi import scenario2dec_nday_average
import numpy as np

start_time = 0.0
end_time = 40.
logging_step_size = 1.0
observed_params = ["I_state"]
intervention_time = torch.tensor(1.)
intervened_params = "p_cbeta"
p_cbeta_current = 0.35
initial_guess_interventions = 0.15
bounds_interventions = [[0.1], [0.5]]

risk_bound = 300.
qoi = lambda x: scenario2dec_nday_average(x, observed_params, 1)
objfun = lambda x: np.abs(p_cbeta_current-x)
static_parameter_interventions = {intervention_time: intervened_params}

opt_result = pyciemss.optimize(model3, end_time, logging_step_size, qoi, risk_bound, static_parameter_interventions, objfun, 
                               initial_guess_interventions=initial_guess_interventions, bounds_interventions=bounds_interventions, 
                               start_time=0.0, n_samples_ouu=int(1e2), maxiter=1, maxfeval=20, solver_method="euler")
print(f'Optimal policy for intervening on {static_parameter_interventions[list(static_parameter_interventions.keys())[0]]} is {opt_result["policy"]:.2f}.')

 40%|████      | 16/40 [01:48<02:43,  6.81s/it]

Optimal policy for intervening on p_cbeta is 0.22.


#### Sample using optimal policy as intervention

In [14]:
num_samples = 100
result = pyciemss.sample(model3, end_time, logging_step_size, num_samples, start_time=start_time, 
                         static_parameter_interventions={intervention_time: {intervened_params: torch.tensor(opt_result["policy"])}}, 
                         solver_method="euler")
result["data"]

,timepoint_id,sample_id,persistent_p_cbeta_param,persistent_p_tr_param,I_state_state,R_state_state,S_state_state
0,0,0,0.349256,12.813910,1.270867,0.078040,999.651123
1,1,0,0.349256,12.813910,1.455598,0.177219,999.367188
2,2,0,0.349256,12.813910,1.667089,0.290814,999.042114
3,3,0,0.349256,12.813910,1.909187,0.420914,998.669922
4,4,0,0.349256,12.813910,2.186285,0.569907,998.243835
...,...,...,...,...,...,...,...
3895,34,99,0.313551,17.372927,169.823944,69.390114,761.786011
3896,35,99,0.313551,17.372927,188.959778,79.165321,732.874939
3897,36,99,0.313551,17.372927,209.030991,90.042000,701.927063
3898,37,99,0.313551,17.372927,229.788483,102.073997,669.137573
